In [210]:
import sys
sys.path.append('..')

import os
import glob
from tqdm import tqdm_notebook as tqdm

from data.bioasq_dataset import BioASQDataset
from utilities.file_utils import Utils as utils
import numpy as np
from collections import Counter
import re
import random

from datasets import load_dataset
from utilities import data_helpers as dh
from IPython.display import HTML as html_print


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## BioAQS dataset stats

Due we don't have access to the ground truth span for test dataset, we are splitting the train set into train/dev/test. For our test set we are using it as second test set more for qualitive results see if it can find the answer even though those examples where not meant for extractive QA like the examples in the original train set. 

In [260]:
dataset = BioASQDataset('train', notebook = True)

Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-train-factoid-5b.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-train-factoid-4b.json


In [261]:
def print_dataset_stats(all_ques, all_ans, all_ctx, total_examples):
    
    all_ques_numwords = [len(q.split()) for q in list(set(all_ques))]
    all_ans_numwords = [len(q.split()) for q in list(set(all_ans))]
    all_ctx_numwords = [len(q.split()) for q in list(set(all_ctx))]
    
    print('Dataset Stats size {}:\n\tTotal questions {}, unique questions {}\n\tTotal answers {}, unique answers: {}\n\tTotal context {} unique context: {}'.format(
            total_examples, 
            len(all_ques), len(list(set(all_ques))),
            len(all_ans), len(list(set(all_ans))),
            len(all_ctx), len(list(set(all_ctx)))))

    print('Questions stats:\n\tmax words {}\n\tmin words: {}\n\tmedian words: {}\n\tmean words:{}'.format(
            np.max(all_ques_numwords), 
            np.min(all_ques_numwords), 
            np.median(all_ques_numwords),
            np.mean(all_ques_numwords)))

    print('Answers stats:\n\tmax words {}\n\tmin words: {}\n\tmedian words: {}\n\tmean words:{}'.format(
            np.max(all_ans_numwords), 
            np.min(all_ans_numwords), 
            np.median(all_ans_numwords),
            np.mean(all_ans_numwords)))

    print('Context stats:\n\tmax words {}\n\tmin words: {}\n\tmedian words: {}\n\tmean words:{}'.format(
            np.max(all_ctx_numwords), 
            np.min(all_ctx_numwords), 
            np.median(all_ctx_numwords),
            np.mean(all_ctx_numwords)))
    
    


In [262]:
out = dataset[0]

In [263]:
print('Stats for BioASQ Dataset')
total_examples = len(dataset)
all_ques = [d[0] for d in dataset]
all_ans = [d[1] for d in dataset]
all_span = [d[2] for d in dataset]
all_ctx = [d[3] for d in dataset]
print_dataset_stats(all_ques, all_ans, all_ctx, total_examples)

Stats for BioASQ Dataset
Dataset Stats size 4957:
	Total questions 4957, unique questions 326
	Total answers 4957, unique answers: 487
	Total context 4957 unique context: 1918
Questions stats:
	max words 23
	min words: 3
	median words: 9.0
	mean words:9.60122699386503
Answers stats:
	max words 40
	min words: 1
	median words: 1.0
	mean words:2.515400410677618
Context stats:
	max words 675
	min words: 22
	median words: 217.0
	mean words:218.40302398331596


In [242]:
def show_one_example(all_ques, all_ans, all_ctx, span, index = None):
    
    if index == None:
        index = random.sample(range(len(all_ques)), 1)[0]
        
    start, end = span[index][0], span[index][1]
    context = all_ctx[index]
    context_text = highlight_ans(start, end, context)
    
    #display_text = 'Question: {}\nContext: {}'.format(all_ques[index], context_text)
    display_text = 'Question: \n{}\nAnswer: {}'.format(all_ques[index], all_ans[index])
    
    return display_text, context_text


In [243]:
def highlight_ans(start, end, context):
    
    template = '<html><body><span style=\"background-color: rgba(0, 255, 0, 0.8);\">{}</font></span></body></html>'
    
    before_span = context[:start] + ' '
    after_span = context[end:] + ' '
    color_area = template.format(context[start:end])
    
    return before_span + color_area + after_span

In [244]:
display_ques, display_example = show_one_example(all_ques, all_ans, all_ctx, all_span)

print(display_ques)
print('Context:')
html_print(display_example)

Question: 
In which kingdom do microsporidia belong, according to their current classification scheme?
Answer: fungi
Context:
